# 1. Inference on Synthetic data

Author: [Marc Lelarge](https://www.di.ens.fr/~lelarge/)[Joseph Assouline]((https://www.linkedin.com/in/joseph-a-82b9bab/)

Date: 04/13

In this notebook, we test our approach on synthetic data.

The problem can be described as follows: we are given a familly of ODEs $y'=h_\theta(y,t)$, where the function $h$ is parametrized by the parameter $\theta$ and a trajectory $z$, the problem is to find the best value of $\theta$ such that $z'\approx h_{\theta}(z,t)$. In order to find the value of $\theta$, we follow an optimization approach using backpropagation through an ODE solver based on the tool developed in [Neural Ordinary Differential Equations](https://arxiv.org/abs/1806.07366). Namely, for a distance function $D$ on $\mathbb{R}^d$, we define $L = D(y_\theta -z)$ where $y_\theta$ is the solution of the ODE $y'=h_{\theta}(y,t)$ and we minimize the loss $L$ with respect to $\theta$ with SGD.

Here, to test this approach, we choose a parameter $\theta$ and integrate the ODE to get the trajectory $z$. We show that based on $z$, we are able to retrieve the parameter $\theta$.

In [99]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(26);
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from model_epidemio import covid_data
from model_epidemio import ihd_fit
import datetime
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## a. the IHDS Model 

We propose a simple version of the SIR Model at the start of the epidemics. In this case, all the population is susceptible.

The standard SIR model is given by the equations:
\begin{eqnarray}
\dot{S}(t) &=& -\beta S(t) I(t)\\
\dot{I}(t) &=& \beta S(t) I(t) - \gamma I(t) -\nu I(t)\\
\dot{R}(t) &=& \gamma I(t)\\
\dot{D}(t) &=& \nu I(t)
\end{eqnarray}
where $S(t)$, $I(t)$, $R(t)$ and $D(t)$ are, respectively, the fractions of susceptible, infectious, recovered and deceased individuals at time $t$. $\beta$ is the contagion rate, $\gamma$ is the recovery rate and $\nu$ is the death rate.

In the early stage of the epidemics, we make the approximation $S(t) \approx 1$ so that the second equation simplifies to $\dot{I}(t) = \beta I(t) - \gamma I(t) -\nu I(t)$.

In a second time when we have fit the right parameters mostly the time we take account of the susceptible to modelize effect of a go back to some normality.

We make two other modifications:
- the contagion rate will depend on time $\beta(t)$
- we add a sub-category of the population $H(t)$, the numbers of individuals in hospital at time $t$. We assume that all deceased individuals are first in a hospital.

We obtain the IHD model given by the equations:
\begin{eqnarray}
\dot{I}(t) &=& \beta(t) I(t) -\gamma I(t)-\nu I(t)\\
\dot{R}(t) &=& \gamma I(t)\\
\dot{H}(t) &=& \nu I(t) - \gamma H(t) - \lambda H(t)\\
\dot{D}(t) &=& \lambda H(t)
\end{eqnarray}
note that the recovered individuals can be ignored to compute $I(t),H(t)$ and $D(t)$.

In practice, we will prametrize the function $\beta(t)$ as follows for IHDS model:
$$\beta(t) = \beta_1 +\delta \sigma(t-\tau)+\delta_1 \sigma(t-\rho)$$
where $\sigma(.)$ is the sigmoid function.

We will prametrize the function $\beta(t)$ as follows for IHD model:
$$\beta(t) = \beta_1 +\delta \sigma(t-\tau)$$
where $\sigma(.)$ is the sigmoid function.

The motivation for using this specific function is to understand the impact of the lock-down on the epidemics. We expect the contagious rate to decrease under lock-down so that $\tau$ should be interpreted as the time when social distancing is implemented resulting in a drop in the contagious rate. And after the lockdown recover a bit as the activity restarts

In [15]:
size = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
true_init = torch.tensor([[0.01,0., 0.]])
t = torch.linspace(0., size-1, size)
name_parms = ['beta', 'delta','gamma','nu','lambda']
parms = torch.tensor([0.15,-0.10,0.05,0.015,0.03])
start = torch.tensor([30.],device=device)
ihd_synt = ihd_fit.IHD_model(parms,start)

$\beta_1=0.15, \delta = -0.1, \gamma = 0.05, \nu = 0.015, \lambda = 0.03$

$\tau = 30 $

$I(0)=0.01, H(0)=0, D(0)=0, S(0)=1$,

In [16]:
y_synt = ihd_fit.predic_ode(ihd_synt, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [17]:
Infected = go.Scatter(x=list(t),
                      y=list(y_synt[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title="population fraction"))
data = [Infected, Hospitalized, Deceased]
fig = go.Figure(data=data, layout=layout)
fig.show()


In [21]:
size = 250
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
true_init = torch.tensor([[0.01,0., 0.,1.,0]])
t = torch.linspace(0., size-1, size)
name_parms = ['beta', 'delta','delta_1','gamma','nu','lambda']
parms = torch.tensor([0.15,-0.10,0.05,0.05,0.015,0.03])
start = torch.tensor([50.],device=device)
end =start+60
ihds_synt = ihd_fit.IHDS_model(parms,start,end)

$\beta_1=0.15, \delta = -0.1, \delta_1 = -0.05,\gamma = 0.05, \nu = 0.015, \lambda = 0.03$

$\tau = 50, \tau_1=110$ 

$I(0)=0.01, H(0)=0, D(0)=0, S(0)=1$,

In [22]:
y_synt_s = ihd_fit.predic_ode(ihds_synt, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [23]:
Susceptible= go.Scatter(x=list(t),
                      y=list(y_synt_s[:,3]),
                      name='Susceptible',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Infected = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Recovered = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,4]),
                      name='recovered',
                      mode='lines',
                      marker=dict(color='#ffcdd1'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt_s[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title="population fraction"))
data = [Infected, Hospitalized, Deceased]
fig = go.Figure(data=data, layout=layout)
fig.show()
data = [Susceptible,Recovered]
fig = go.Figure(data=data, layout=layout)
fig.show()

Integration of the ODE: Infected = Blue, Hospital = green, Deaths = red

These trajectories are obtained by integrating the IHD model. To a 'non-specialist', these curves seem plausible: we observe the typical exponential growth at the start of the epidemics and when the R0 goes below one (around time 50), the number of Infected starts to decrease, as well as the number of individuals in hospital.

## b. Inference problem

Now given the trajectory depicted above, we try to recover the parameters of the model.

### IHD Model

In [43]:
size = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
true_init = torch.tensor([[0.01,0., 0.]])
t = torch.linspace(0., size-1, size)
name_parms = ['beta', 'delta','gamma','nu','lambda']
parms = torch.tensor([0.15,-0.10,0.05,0.015,0.03])
start = torch.tensor([30.],device=device)
ihd_synt = ihd_fit.IHD_model(parms,start)
y_synt = ihd_fit.predic_ode(ihd_synt, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [44]:
parms_fit = torch.tensor([0.15,-0.05,0.05,0.05,0.05])
time_fit = torch.tensor([50.])
ihd_time = ihd_fit.IHD_model(parms_fit,time_fit)

In [45]:
list(ihd_time.parameters())

[Parameter containing:
 tensor(0.1500, requires_grad=True),
 Parameter containing:
 tensor(-0.0500, requires_grad=True),
 Parameter containing:
 tensor(0.0500, requires_grad=True),
 Parameter containing:
 tensor(0.0500, requires_grad=True),
 Parameter containing:
 tensor(0.0500, requires_grad=True),
 Parameter containing:
 tensor([50.], requires_grad=True)]

In [46]:
optimizer_time = optim.Adam([{'params': [ihd_time.b1, ihd_time.b2, ihd_time.g, ihd_time.nu, ihd_time.l]},
                                {'params': ihd_time.start, 'lr' : 1}], lr=1e-3)
criterion = nn.MSELoss()

best_loss, best_parms = ihd_fit.trainig(ihd_time, init=true_init.cuda(), t=t.cuda(), optimizer=optimizer_time,
                                        criterion=criterion,niters=600,data=torch.from_numpy(y_synt).cuda(), data_cat={"I": 0, "H": 1, "D": 2})

[0 1 2]
1 0.002284388989210129 [0.14900000393390656, -0.050999999046325684, 0.050999999046325684, 0.050999999046325684, 0.049000002443790436, 49.00004196166992]
10 0.0008488752646371722 [0.14447864890098572, -0.05552009865641594, 0.05574614927172661, 0.054780442267656326, 0.04178265109658241, 44.273841857910156]
20 0.0007690694183111191 [0.14509406685829163, -0.055255476385354996, 0.05555032566189766, 0.052843306213617325, 0.036722294986248016, 44.71794891357422]
30 0.0005378302885219455 [0.14747130870819092, -0.053477007895708084, 0.05371362343430519, 0.04888007044792175, 0.03166121616959572, 46.745994567871094]
40 0.0004552542814053595 [0.1470925211906433, -0.055140599608421326, 0.05477554723620415, 0.04733246564865112, 0.024511568248271942, 45.186004638671875]
50 0.0004081172519363463 [0.1472802460193634, -0.05674397200345993, 0.055336859077215195, 0.045112237334251404, 0.01936561055481434, 43.90638732910156]
60 0.00036267389077693224 [0.14788870513439178, -0.058580055832862854, 0.0

540 9.0569818311792e-09 [0.15500403940677643, -0.09996794164180756, 0.054324112832546234, 0.015699828043580055, 0.02980472892522812, 30.004274368286133]
550 9.05206487544774e-09 [0.15500333905220032, -0.09996869415044785, 0.054322633892297745, 0.015699870884418488, 0.02980390563607216, 30.004032135009766]
560 9.047274929230298e-09 [0.15500253438949585, -0.0999692976474762, 0.054321255534887314, 0.015699872747063637, 0.02980327419936657, 30.00384521484375]
570 9.043616522319553e-09 [0.15500164031982422, -0.09996974468231201, 0.05431995913386345, 0.015699828043580055, 0.02980279177427292, 30.00370979309082]
580 9.03894381565351e-09 [0.15500065684318542, -0.09997010976076126, 0.05431871488690376, 0.015699755400419235, 0.029802434146404266, 30.003604888916016]
590 9.03491592652017e-09 [0.15499961376190186, -0.09997038543224335, 0.05431750789284706, 0.015699652954936028, 0.029802173376083374, 30.00352668762207]
600 9.030162395617936e-09 [0.1549985706806183, -0.09997060149908066, 0.054316334

In [47]:
best_loss

9.030162395617936e-09

In [48]:
parms_inf = torch.cat([p.data.unsqueeze(0) for p in best_parms[:-1]], 0)
time_inf = best_parms[-1].data
ihd_inf = ihd_fit.IHD_model(parms_inf, time_inf)

In [49]:
y_inf = ihd_fit.predic_ode(ihd_inf, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [50]:
Infected = go.Scatter(x=list(t),
                      y=list(y_synt[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Infected_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,0]),
                      name='Infected prediction',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Hospitalized_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,1]),
                          name="Hospitalized prediction",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
Deceased_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,2]),
                      name="Deceased prediction",
                      mode='markers',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title=" % population"))
data = [Infected,Infected_pred, Hospitalized,Hospitalized_pred, Deceased,Deceased_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()

Infected = Blue, Hospital = green, Deaths = red; dashed = true, plain = estimated.

We see that we have a good fit of the trajectories. Below, we look at the estimation of the parameters directly.

In [51]:
for i,p in enumerate(best_parms):
    try: 
        print(name_parms[i],',true: ', parms[i].item(), ',evaluated: ', p.data.item(), 'error: %',(p.data.item()-parms[i].item())/parms[i].item()*100)
    except:
        print('time',',true: ', start.item(), ',evaluated: ', p.data.item(), 'error: %',(start.item()-p.data.item())/p.data.item()*100)

beta ,true:  0.15000000596046448 ,evaluated:  0.1549985706806183 error: % 3.3323763476858
delta ,true:  -0.10000000149011612 ,evaluated:  -0.09997060149908066 error: % -0.029399990597367427
gamma ,true:  0.05000000074505806 ,evaluated:  0.054316334426403046 error: % 8.632667234053205
nu ,true:  0.014999999664723873 ,evaluated:  0.015699541196227074 error: % 4.663610314261151
lambda ,true:  0.029999999329447746 ,evaluated:  0.0298019852489233 error: % -0.6600469498346809
time ,true:  30.0 ,evaluated:  30.003448486328125 error: % -0.011493633239180474


- We see that the switching time is correctly estimated
- The contagious ( beta) receovered(gamma) hospitalized (nu)  rates are over estimated.
- The death rate is underestiua

### IHDS Model

In [34]:
size = 250
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
true_init = torch.tensor([[0.01,0., 0.,1.,0]])
t = torch.linspace(0., size-1, size)
name_parms = ['beta', 'delta','delta_1','gamma','nu','lambda']
parms = torch.tensor([0.15,-0.10,0.05,0.05,0.015,0.03])
start = torch.tensor([50.],device=device)
end =start+60
ihds_synt = ihd_fit.IHDS_model(parms,start,end)

In [35]:
parms_fit = torch.tensor([0.1,-0.10,0.025,0.025,0.03,0.015])
start_fit = torch.tensor([50.])
end_fit = start_fit + 40 
ihd_time = ihd_fit.IHDS_model(parms, start_fit, end_fit)

In [36]:
optimizer_time = optim.Adam([{'params': [ihd_time.b1, ihd_time.b2, ihd_time.g, ihd_time.nu, ihd_time.l]},
                                {'params': [ihd_time.start,ihd_time.end], 'lr' : 1}], lr=1e-5)
criterion = nn.MSELoss()

best_loss, best_parms = ihd_fit.trainig(ihd_time, init=true_init.cuda(), t=t.cuda(), optimizer=optimizer_time,
                                        criterion=criterion,niters=600,data=torch.from_numpy(y_synt_s).cuda(), data_cat={"I": 0, "H": 1, "D": 2, "S":3, "R":4})

[0 1 2 3 4]
1 7.838218152755871e-05 [0.1499900072813034, -0.10001000016927719, 0.05000000074505806, 0.05000999942421913, 0.01500999927520752, 0.029989998787641525, 49.0001220703125, 90.99913787841797]
10 2.435814167256467e-05 [0.14999279379844666, -0.10002943128347397, 0.05000000074505806, 0.05001549422740936, 0.015015139244496822, 0.029987944290041924, 49.651058197021484, 94.957763671875]
20 5.3640246733266395e-06 [0.1499907225370407, -0.10005815327167511, 0.05000000074505806, 0.050027281045913696, 0.01502606924623251, 0.029980512335896492, 49.84651565551758, 99.77035522460938]
30 2.2339354472933337e-06 [0.1499871015548706, -0.1000836044549942, 0.05000000074505806, 0.0500386618077755, 0.015036295168101788, 0.029971890151500702, 49.77157211303711, 103.58677673339844]
40 1.0176477189816069e-06 [0.14998631179332733, -0.1001005694270134, 0.05000000074505806, 0.050045114010572433, 0.01504141092300415, 0.029966041445732117, 49.84535217285156, 106.0975341796875]
50 2.3047232389217243e-07 [0.

430 8.127510886524192e-10 [0.15001997351646423, -0.10011328011751175, 0.05000000074505806, 0.05002285912632942, 0.015007211826741695, 0.030010292306542397, 50.023902893066406, 109.68278503417969]
440 7.943694591006079e-10 [0.15002012252807617, -0.10011283308267593, 0.05000000074505806, 0.050022393465042114, 0.015007122419774532, 0.03000991977751255, 50.02328109741211, 109.68560791015625]
450 7.776188581942733e-10 [0.15002015233039856, -0.10011238604784012, 0.05000000074505806, 0.0500219464302063, 0.01500704139471054, 0.030009547248482704, 50.02274703979492, 109.68831634521484]
460 7.618039532530929e-10 [0.15002015233039856, -0.1001119390130043, 0.05000000074505806, 0.050021518021821976, 0.015006966888904572, 0.030009180307388306, 50.0223274230957, 109.69087219238281]
470 7.483833552868191e-10 [0.15002015233039856, -0.10011149197816849, 0.05000000074505806, 0.050021108239889145, 0.01500689797103405, 0.030008824542164803, 50.02190017700195, 109.6933364868164]
480 7.341065533239544e-10 [0

In [37]:
best_loss

6.123044293815383e-10

In [38]:
best_parms

[Parameter containing:
 tensor(0.1500, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(-0.1001, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(0.0500, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(0.0500, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(0.0150, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor(0.0300, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([50.0180], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([109.7208], device='cuda:0', requires_grad=True)]

In [39]:
parms_inf = torch.cat([p.data.unsqueeze(0) for p in best_parms[:-2]], 0)
start_best = best_parms[-2].data
end_best = best_parms[-1].data
ihd_inf = ihd_fit.IHDS_model(parms_inf, start_best, end_best)

In [40]:
y_inf = ihd_fit.predic_ode(ihd_inf, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [41]:
Infected = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Infected_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,0]),
                      name='Infected prediction',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt_s[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Hospitalized_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,1]),
                          name="Hospitalized prediction",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
Deceased_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,2]),
                      name="Deceased prediction",
                      mode='markers',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title=" % population"))
data = [Infected,Infected_pred, Hospitalized,Hospitalized_pred, Deceased,Deceased_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()
Susceptible = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,3]),
                      name='Susceptible',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Susceptible_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,3]),
                      name='Susceptible prediction',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Recovered = go.Scatter(x=list(t),
                          y=list(y_synt_s[:,4]),
                          name="Recovered",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Recovered_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,4]),
                          name="Recovered pred",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
data = [Susceptible,Susceptible_pred, Recovered, Recovered_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [42]:
for i,p in enumerate(best_parms):
    try: 
        print(name_parms[i],',true: ', parms[i].item(), ',evaluated: ', p.data.item(), 'error: %',(p.data.item()-parms[i].item())/parms[i].item()*100)
    except:
        if i == len(best_parms)-2:
            print('start',',true: ', start.item(), ',evaluated: ', p.data.item(), 'error: %',(start.item()-p.data.item())/p.data.item()*100)
        elif i==len(best_parms)-1:
            print('end',',true: ', end.item(), ',evaluated: ', p.data.item(), 'error: %',(end.item()-p.data.item())/p.data.item()*100)

beta ,true:  0.15000000596046448 ,evaluated:  0.15001919865608215 error: % 0.012795129903351072
delta ,true:  -0.10000000149011612 ,evaluated:  -0.10010503977537155 error: % 0.10503828369023974
delta_1 ,true:  0.05000000074505806 ,evaluated:  0.05000000074505806 error: % 0.0
gamma ,true:  0.05000000074505806 ,evaluated:  0.05001699551939964 error: % 0.03398954817668277
nu ,true:  0.014999999664723873 ,evaluated:  0.015006187371909618 error: % 0.04125138216034184
lambda ,true:  0.029999999329447746 ,evaluated:  0.030005307868123055 error: % 0.01769512931321105
start ,true:  50.0 ,evaluated:  50.01803207397461 error: % -0.0360511464104399
end ,true:  110.0 ,evaluated:  109.72077178955078 error: % 0.25448983441785356


## c.2 missing infected
We now consider a case where the number of infected $I(t)$ is not available (for example, because the population is not tested). We only have acces to the number of individuals in hospital $H(t)$ and deceased individuals $D(t)$.

### IHD Model

In [82]:
parms_fit = torch.tensor([0.10,-0.05,0.1,0.03,0.05])
time_fit = torch.tensor([50.])
ihd_time = ihd_fit.IHD_model(parms_fit,time_fit)

In [83]:
optimizer_time = optim.Adam([{'params': [ihd_time.b1, ihd_time.b2, ihd_time.g, ihd_time.nu, ihd_time.l]},
                                {'params': ihd_time.start, 'lr' : 1}], lr=1e-3)
criterion = nn.MSELoss()

best_loss, best_parms = ihd_fit.trainig(ihd_time, init=true_init.cuda(), t=t.cuda(), optimizer=optimizer_time,
                                        criterion=criterion,niters=600,data=torch.from_numpy(y_synt[:,1:3]).cuda(), data_cat={"H": 1, "D": 2})

[1 2]
1 0.0002180874434998259 [0.10099998861551285, -0.04900036007165909, 0.0990000069141388, 0.03099997341632843, 0.05099997669458389, 50.93826675415039]
10 0.00018333572370465845 [0.11015895009040833, -0.03986162319779396, 0.08984437584877014, 0.03874344006180763, 0.060080841183662415, 59.53141403198242]
20 9.693788888398558e-05 [0.12114550173282623, -0.02890537865459919, 0.0788763090968132, 0.03419063240289688, 0.07045505195856094, 69.85848236083984]
30 5.0032227591145784e-05 [0.12768930196762085, -0.023815516382455826, 0.07222773134708405, 0.029289979487657547, 0.07503776252269745, 75.16387176513672]
40 3.8639256672468036e-05 [0.12535905838012695, -0.02872079238295555, 0.07431644201278687, 0.033319320529699326, 0.0699930414557457, 71.05337524414062]
50 2.9799588446621783e-05 [0.12806107103824615, -0.028782058507204056, 0.07147997617721558, 0.03164930269122124, 0.06669163703918457, 71.4252700805664]
60 2.4566226784372702e-05 [0.12766875326633453, -0.03329688310623169, 0.071755722165

540 8.818933139309593e-08 [0.14293208718299866, -0.0780601054430008, 0.055538006126880646, 0.02449154295027256, 0.02966090850532055, 33.25093460083008]
550 8.484298774646959e-08 [0.14300847053527832, -0.07839589565992355, 0.05546695366501808, 0.02429901249706745, 0.029668394476175308, 33.1934928894043]
560 8.163427622775998e-08 [0.1430845856666565, -0.07872665673494339, 0.05539607256650925, 0.024111200124025345, 0.029675642028450966, 33.13706588745117]
570 7.855649641896889e-08 [0.1431603729724884, -0.0790524035692215, 0.055325400084257126, 0.023927979171276093, 0.029682660475373268, 33.08168029785156]
580 7.560495163261294e-08 [0.14323581755161285, -0.07937319576740265, 0.0552549846470356, 0.023749226704239845, 0.02968946099281311, 33.027286529541016]
590 7.277294855612126e-08 [0.14331090450286865, -0.07968906313180923, 0.05518486723303795, 0.023574814200401306, 0.02969605289399624, 32.9738883972168]
600 7.00567781564132e-08 [0.14338554441928864, -0.07999999821186066, 0.05511508509516

In [84]:
best_loss

7.00567781564132e-08

In [88]:
parms_inf = torch.cat([p.data.unsqueeze(0) for p in best_parms[:-1]], 0)
time_inf = best_parms[-1].data
ihd_inf = ihd_fit.IHD_model(parms_inf, time_inf)
y_inf = ihd_fit.predic_ode(ihd_inf, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [89]:
Infected = go.Scatter(x=list(t),
                      y=list(y_synt[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Infected_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,0]),
                      name='Infected prediction',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Hospitalized_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,1]),
                          name="Hospitalized prediction",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
Deceased_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,2]),
                      name="Deceased prediction",
                      mode='markers',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title=" % population"))
data = [Infected,Infected_pred, Hospitalized,Hospitalized_pred, Deceased,Deceased_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [90]:
for i,p in enumerate(best_parms):
    try: 
        print(name_parms[i],',true: ', parms[i].item(), ',evaluated: ', p.data.item(), 'error: %',(p.data.item()-parms[i].item())/parms[i].item()*100)
    except:
        print('time',',true: ', start.item(), ',evaluated: ', p.data.item(), 'error: %',(start.item()-p.data.item())/p.data.item()*100)

beta ,true:  0.15000000596046448 ,evaluated:  0.14338554441928864 error: % -4.409640852227177
delta ,true:  -0.10000000149011612 ,evaluated:  -0.07999999821186066 error: % -20.000002980232196
gamma ,true:  0.05000000074505806 ,evaluated:  0.05511508509516716 error: % 10.230168547776811
nu ,true:  0.014999999664723873 ,evaluated:  0.023404620587825775 error: % 56.03080740639882
lambda ,true:  0.029999999329447746 ,evaluated:  0.029702458530664444 error: % -0.9918026847795253
time ,true:  30.0 ,evaluated:  32.92158126831055 error: % -8.874364947721343


We see that we obtain much better results here. Although we do not observe the curve of infected individuals, we are able to get a rough estimate of it.

### IHDS Model (recovered and susceptible  are missing)

In [91]:
size = 250
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
true_init = torch.tensor([[0.01,0., 0.,1.,0]])
t = torch.linspace(0., size-1, size)
name_parms = ['beta', 'delta','delta_1','gamma','nu','lambda']
parms = torch.tensor([0.15,-0.10,0.05,0.05,0.015,0.03])
start = torch.tensor([50.],device=device)
end =start+60
ihds_synt = ihd_fit.IHDS_model(parms,start,end)

In [92]:
parms_fit = torch.tensor([0.2,-0.15,0.025,0.025,0.03,0.015])
start_fit = torch.tensor([50.])
end_fit = start_fit + 40 
ihd_time = ihd_fit.IHDS_model(parms, start_fit, end_fit)

In [93]:
optimizer_time = optim.Adam([{'params': [ihd_time.b1, ihd_time.b2, ihd_time.g, ihd_time.nu, ihd_time.l]},
                                {'params': [ihd_time.start,ihd_time.end], 'lr' : 1}], lr=1e-5)
criterion = nn.MSELoss()

best_loss, best_parms = ihd_fit.trainig(ihd_time, init=true_init.cuda(), t=t.cuda(), optimizer=optimizer_time,
                                        criterion=criterion,niters=600,data=torch.from_numpy(y_synt_s[:,0:3]).cuda(), data_cat={"I": 0, "H": 1, "D": 2})

[0 1 2]
1 2.67515633822768e-06 [0.1499900072813034, -0.10001000016927719, 0.05000000074505806, 0.05000999942421913, 0.014990000054240227, 0.029989998787641525, 49.01980209350586, 90.97411346435547]
10 8.878249104782299e-07 [0.14997830986976624, -0.10008656978607178, 0.05000000074505806, 0.050063036382198334, 0.014952942728996277, 0.02991458587348461, 49.65843963623047, 99.17018127441406]
20 1.7117518780196406e-07 [0.14997468888759613, -0.10014548152685165, 0.05000000074505806, 0.050097908824682236, 0.014952825382351875, 0.029874499887228012, 49.86412048339844, 105.66285705566406]
30 4.81122199857964e-08 [0.1499902755022049, -0.10017208009958267, 0.05000000074505806, 0.0501013845205307, 0.01499001495540142, 0.029887957498431206, 49.98986053466797, 109.0550308227539]
40 2.3602026999469672e-08 [0.15001840889453888, -0.1001753881573677, 0.05000000074505806, 0.050085023045539856, 0.015029684640467167, 0.029926136136054993, 50.0349235534668, 110.29683685302734]
50 1.2913803359992926e-08 [0.1

430 1.143858792407304e-11 [0.15002383291721344, -0.09999245405197144, 0.05000000074505806, 0.05001544952392578, 0.015002008527517319, 0.030002623796463013, 50.003990173339844, 109.94764709472656]
440 9.966594390065087e-12 [0.15002334117889404, -0.09999103099107742, 0.05000000074505806, 0.050015151500701904, 0.015001971274614334, 0.03000251017510891, 50.003761291503906, 109.95100402832031]
450 8.704197952680293e-12 [0.15002289414405823, -0.09998972713947296, 0.05000000074505806, 0.050014857202768326, 0.015001947060227394, 0.030002381652593613, 50.00353240966797, 109.95411682128906]
460 7.661907566736126e-12 [0.1500224471092224, -0.09998851269483566, 0.05000000074505806, 0.050014596432447433, 0.015001928433775902, 0.03000224381685257, 50.00333786010742, 109.95704650878906]
470 6.743731441327672e-12 [0.1500220000743866, -0.09998741000890732, 0.05000000074505806, 0.05001433566212654, 0.01500190980732441, 0.030002105981111526, 50.00318145751953, 109.95970153808594]
480 5.9224873269481115e-1

In [95]:
parms_inf = torch.cat([p.data.unsqueeze(0) for p in best_parms[:-2]], 0)
start_best = best_parms[-2].data
end_best = best_parms[-1].data
ihd_inf = ihd_fit.IHDS_model(parms_inf, start_best, end_best)

In [96]:
y_inf = ihd_fit.predic_ode(ihd_inf, true_init.cuda(),t.cuda()).detach().cpu().numpy()

In [97]:
Infected = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Infected_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,0]),
                      name='Infected prediction',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt_s[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Hospitalized_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,1]),
                          name="Hospitalized prediction",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
Deceased_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,2]),
                      name="Deceased prediction",
                      mode='markers',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title=" % population"))
data = [Infected,Infected_pred, Hospitalized,Hospitalized_pred, Deceased,Deceased_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()
Susceptible = go.Scatter(x=list(t),
                      y=list(y_synt_s[:,3]),
                      name='Susceptible',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Susceptible_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,3]),
                      name='Susceptible prediction',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Recovered = go.Scatter(x=list(t),
                          y=list(y_synt_s[:,4]),
                          name="Recovered",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Recovered_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,4]),
                          name="Recovered pred",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
data = [Susceptible,Susceptible_pred, Recovered, Recovered_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [98]:
for i,p in enumerate(best_parms):
    try: 
        print(name_parms[i],',true: ', parms[i].item(), ',evaluated: ', p.data.item(), 'error: %',(p.data.item()-parms[i].item())/parms[i].item()*100)
    except:
        if i == len(best_parms)-2:
            print('start',',true: ', start.item(), ',evaluated: ', p.data.item(), 'error: %',(start.item()-p.data.item())/p.data.item()*100)
        elif i==len(best_parms)-1:
            print('end',',true: ', end.item(), ',evaluated: ', p.data.item(), 'error: %',(end.item()-p.data.item())/p.data.item()*100)

beta ,true:  0.15000000596046448 ,evaluated:  0.15001846849918365 error: % 0.012308358657027934
delta ,true:  -0.10000000149011612 ,evaluated:  -0.09997972100973129 error: % -0.02028048008262396
delta_1 ,true:  0.05000000074505806 ,evaluated:  0.05000000074505806 error: % 0.0
gamma ,true:  0.05000000074505806 ,evaluated:  0.0500120148062706 error: % 0.024028122067032427
nu ,true:  0.014999999664723873 ,evaluated:  0.015001822263002396 error: % 0.012150655461738257
lambda ,true:  0.029999999329447746 ,evaluated:  0.030000731348991394 error: % 0.002440065200032261
start ,true:  50.0 ,evaluated:  50.00175476074219 error: % -0.0035093983212869423
end ,true:  110.0 ,evaluated:  109.98005676269531 error: % 0.018133503374815632


# Real data

## Loading data

In [100]:
df = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/full_data.csv', 
                     index_col='date', parse_dates=True)
df = df.rename(columns={'location': 'country', 'total_cases': 'confirmed', 'total_deaths': 'deaths'}) 

In [101]:
df.head()

,country,new_cases,new_deaths,confirmed,deaths
date,,,,,
2019-12-31,Afghanistan,0,0,0,0
2020-01-01,Afghanistan,0,0,0,0
2020-01-02,Afghanistan,0,0,0,0
2020-01-03,Afghanistan,0,0,0,0
2020-01-04,Afghanistan,0,0,0,0


In [102]:
pop_file = 'https://github.com/datasets/population/raw/master/data/population.csv'
populations_all = pd.read_csv(pop_file)
populations_all = populations_all.sort_values(['Country Name', 'Year']).groupby(['Country Name']).last()
populations_all.loc['Egypt'] = populations_all.loc['Egypt, Arab Rep.'].Value
populations_all.loc['Iran'] = populations_all.loc['Iran, Islamic Rep.'].Value
populations_all.loc['South Korea'] = populations_all.loc['Korea, Rep.'].Value
populations_all.loc['Hong Kong'] = populations_all.loc['Hong Kong SAR, China'].Value
populations_all.loc['Czechia'] = populations_all.loc['Czech Republic'].Value

## France

In [ ]:
df_france = df[df.country == 'France']
scale = 100
fr_pop = populations_all.loc['France']['Value']
death, new_cases = covid_data.create_death_cases(df_france,scale,fr_pop)
delta = 10
init = np.nonzero(new_cases)[0][0]+delta
t, data_death, data_cases = covid_data.make_data(death,new_cases,init,delta)
cases_init = data_cases[0].item()
parms_fit = torch.tensor([0.3,0.05,0.05,0.3])
ihd = ihd_fit.IHD_fit(parms_fit)
true_init = torch.tensor([[cases_init,0., 0.,1.]])
optimizer = optim.Adam(ihd.parameters(), lr=1e-3)
criterion = nn.MSELoss()
best_loss, best_parms = ihd_fit.trainig(ihd, init=true_init, t=t, optimizer=optimizer,
                                        criterion=criterion,niters=200,data=torch.concat([]) data_cat={'I':0,''})
ihd_inf = ihd_fit.get_best_model_simple(best_parms)
y_inf = ihd_fit.predic_ode(ihd_inf, true_init,t)

In [ ]:
Infected = go.Scatter(x=list(t),
                      y=list(y_synt[:,0]),
                      name='Infected',
                      mode='lines',
                      marker=dict(color='#ffcdd2'))
Infected_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,0]),
                      name='Infected',
                      mode='markers',
                      marker=dict(color='#ffcdd2'))
Hospitalized = go.Scatter(x=list(t),
                          y=list(y_synt[:,1]),
                          name="Hospitalized",
                          mode='lines',
                          marker=dict(color='#A2D5F2'))
Hospitalized_pred = go.Scatter(x=list(t),
                          y=list(y_inf[:,1]),
                          name="Hospitalized",
                          mode='markers',
                          marker=dict(color='#A2D5F2'))
Deceased = go.Scatter(x=list(t),
                      y=list(y_synt[:,2]),
                      name="Deceased",
                      mode='lines',
                      marker=dict(color='#59606D'))
Deceased_pred = go.Scatter(x=list(t),
                      y=list(y_inf[:,2]),
                      name="Deceased",
                      mode='markers',
                      marker=dict(color='#59606D'))
layout = go.Layout(title="Model Epidemio",
                    xaxis=dict(title="time from case 0"),
                    yaxis=dict(title=" % population"))
data = [Infected,Infected_pred, Hospitalized,Hospitalized_pred, Deceased,Deceased_pred]
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
initial_date = 
data_hospital = pd.read_csv(
    "https://static.data.gouv.fr/resources/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/20200412-190015/donnees-hospitalieres-covid19-2020-04-12-19h00.csv", sep=";")
df_hospital = data_hospital.loc[data_hospital['sexe']!=0].groupby(['jour']).agg({'hosp' : 'sum', 'rea' : 'sum', 'rad' : 'sum', 'dc':'sum'}).reset_index()
hosiptal_fr =list(df_hospital['hosp'])/fr_pop*scale

In [ ]:
hosiptal_fr =list(df_hospital['hosp'])/fr_pop*scale

In [ ]:
plt.plot(y_inf[-19:,0], 'b')
plt.plot(data_cases[-19:], 'b--')
plt.plot(y_inf[-19:,1], 'g')
plt.plot(hosiptal_fr, 'g--')
plt.plot(y_inf[-19:,2], 'r')
plt.plot(data_death[-19:], 'r--')

## South Korea

## US